In [68]:
from scipy.io.wavfile import read
import scipy.signal as signal
import matplotlib as mpl
from scipy.stats import skew, kurtosis
cmap = mpl.colormaps['Reds']


In [69]:

sampling_rate,data= read(r"C:\Users\Admin\Downloads\baby-crying-01.wav")

C:\Users\Admin\AppData\Local\Temp\ipykernel_27788\3213013389.py:1: WavFileWarning: Chunk (non-data) not understood, skipping it.
  sampling_rate,data= read(r"C:\Users\Admin\Downloads\baby-crying-01.wav")


In [70]:
print(data.shape)

(816000, 2)


In [71]:
wav=data[:,1]

In [72]:
print(type(wav))

<class 'numpy.ndarray'>


In [73]:
plt.plot(wav)
plt.show()

NameError: name 'plt' is not defined

In [74]:
f, t, Zxx = signal.stft(wav, fs=sampling_rate)

In [75]:
print(f.shape)

(129,)


In [76]:
plt.pcolormesh(t, f, np.abs(Zxx), cmap=cmap)

NameError: name 'plt' is not defined

In [77]:
import os
import numpy as np
from scipy.io import wavfile
from scipy.signal import spectrogram
import pandas as pd
import sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from scipy.stats import skew, kurtosis


In [78]:
# Directory containing the infant cry audio files
data_dir_cry = r"D:\infant cry detecting\dev\baby_cry\baby_cry_detection\data\301 - Crying baby"

# List all files in the directory
audio_files_cry = os.listdir(data_dir_cry)

In [79]:
# Directory containing the infant non-cry audio files
data_dir_non_cry = r"D:\infant cry detecting\dev\baby_cry\baby_cry_detection\data\901 - Silence"

# List all files in the directory
audio_files_non_cry = os.listdir(data_dir_non_cry)

In [93]:
# Sample rate and other parameters
fs = 1000  # Sample rate 
nperseg = 256  # Window size for STFT 
noverlap = 128  # Overlap between windows 

# Initialize empty lists for features
features = []
labels = []

# List all files in the directory
#audio_files_cry = os.listdir(data_dir_cry)
#audio_files_non_cry = os.listdir(data_dir_non_cry)

# Loop through each cry audio file
for audio_file in audio_files_cry:
    # Check if the file is a WAV file
    if audio_file.endswith(".wav"):
        # Construct the full path to the audio file
        audio_path = os.path.join(data_dir_cry, audio_file)
        
        # Read the audio file
        fs, audio_signal = wavfile.read(audio_path)

        # Compute STFT
        f, t, Sxx = spectrogram(audio_signal, fs=fs, nperseg=nperseg, noverlap=noverlap)

        # Calculate the 20 features from STFT
        tf_mean = np.mean(Sxx)
        tf_std = np.std(Sxx)
        
        tma = np.max(Sxx, axis=0)
        tma_max = np.max(tma)
        tma_min = np.min(tma)
        tma_mean = np.mean(tma)
        tma_std = np.std(tma)
        tma_skewness = skew(tma)
        tma_kurt = kurtosis(tma)
        
        fma = np.max(Sxx, axis=1)
        fma_max = np.max(fma)
        fma_min = np.min(fma)
        fma_mean = np.mean(fma)
        fma_std = np.std(fma)
        fma_skewness = skew(fma)
        fma_kurt = kurtosis(fma)
        
        fsda = np.std(Sxx, axis=1)
        fsda_max = np.max(fsda)
        fsda_min = np.min(fsda)
        fsda_mean = np.mean(fsda)
        fsda_std = np.std(fsda)
        fsda_skewness = skew(fsda)
        fsda_kurt = kurtosis(fsda)

        # Determine if it's a cry or non-cry sound based on file name or other criteria
        # For example, you can use a naming convention or other metadata to label the data
        is_cry = 1 #if "cry" in audio_file else 0

        # Combine features into a feature vector
        feature_vector = np.array([is_cry, tf_mean, tf_std,
                                   tma_max, tma_min, tma_mean, tma_std, tma_skewness, tma_kurt,
                                   fma_max, fma_min, fma_mean, fma_std, fma_skewness, fma_kurt,
                                   fsda_max, fsda_min, fsda_mean, fsda_std, fsda_skewness, fsda_kurt])

        # Append feature vector and label to lists
        features.append(feature_vector)
        labels.append(is_cry)
        
print(feature_vector)       
      





[1.00000000e+00 1.17209686e-06 1.39945569e-05 1.48291746e-03
 1.98226999e-08 7.38225208e-05 1.28104657e-04 4.08245627e+00
 2.35497094e+01 1.48291746e-03 1.56257155e-13 3.94161580e-05
 1.64077515e-04 6.53207283e+00 4.83976345e+01 1.14580682e-04
 2.14652311e-14 3.29871182e-06 1.28885904e-05 6.30390051e+00
 4.54299877e+01]


In [94]:
# Loop through each audio file in the non-cry directory
for audio_file in audio_files_non_cry:
    # Check if the file is a WAV file
    if audio_file.endswith(".wav"):
        # Construct the full path to the audio file
        audio_path = os.path.join(data_dir_non_cry, audio_file)
        
        # Read the audio file
        fs, audio_signal = wavfile.read(audio_path)

        # Compute STFT
        f, t, Sxx = spectrogram(audio_signal, fs=fs, nperseg=nperseg, noverlap=noverlap)

        # Calculate the 20 features from STFT
        tf_mean = np.mean(Sxx)
        tf_std = np.std(Sxx)
        
        tma = np.max(Sxx, axis=0)
        tma_max = np.max(tma)
        tma_min = np.min(tma)
        tma_mean = np.mean(tma)
        tma_std = np.std(tma)
        tma_skewness = skew(tma)
        tma_kurt = kurtosis(tma)
        
        fma = np.max(Sxx, axis=1)
        fma_max = np.max(fma)
        fma_min = np.min(fma)
        fma_mean = np.mean(fma)
        fma_std = np.std(fma)
        fma_skewness = skew(fma)
        fma_kurt = kurtosis(fma)
        
        fsda = np.std(Sxx, axis=1)
        fsda_max = np.max(fsda)
        fsda_min = np.min(fsda)
        fsda_mean = np.mean(fsda)
        fsda_std = np.std(fsda)
        fsda_skewness = skew(fsda)
        fsda_kurt = kurtosis(fsda)

        # Label non-cry audio as 0
        is_cry = 0

        # Combine features into a feature vector
        feature_vector = np.array([is_cry, tf_mean, tf_std,
                                   tma_max, tma_min, tma_mean, tma_std, tma_skewness, tma_kurt,
                                   fma_max, fma_min, fma_mean, fma_std, fma_skewness, fma_kurt,
                                   fsda_max, fsda_min, fsda_mean, fsda_std, fsda_skewness, fsda_kurt])

        # Append feature vector and label to lists
        features.append(feature_vector)
        labels.append(is_cry)
print(feature_vector)       


[0.00000000e+00 3.47935952e-07 6.85012310e-06 7.70683633e-04
 5.40738156e-08 3.63501713e-05 6.41047372e-05 4.96085391e+00
 3.28480428e+01 7.70683633e-04 5.63178304e-09 1.12078897e-05
 7.99566114e-05 8.26234754e+00 6.96523667e+01 6.13858647e-05
 5.14747689e-10 7.90323156e-07 6.03004128e-06 8.86121682e+00
 8.10637339e+01]


In [95]:
# Create a DataFrame - column names
column_names = ['IsCry', 'TF_Mean', 'TF_Std', 'TMA_Max', 'TMA_Min', 'TMA_Mean', 'TMA_Std', 'TMA_Skewness', 'TMA_Kurtosis',
                'FMA_Max', 'FMA_Min', 'FMA_Mean', 'FMA_Std', 'FMA_Skewness', 'FMA_Kurtosis',
                'FSDA_Max', 'FSDA_Min', 'FSDA_Mean', 'FSDA_Std', 'FSDA_Skewness', 'FSDA_Kurtosis']

df = pd.DataFrame(features, columns=column_names)

# Shuffle the DataFrame
df = df.sample(frac=1, random_state=42).reset_index(drop=True)


# Split the data into training and testing sets (80% train, 20% test)
X = df.drop('IsCry', axis=1)
y = df['IsCry']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



Train a Random Forest Classifier

In [96]:
df.dropna(inplace=True)  # Drops rows with missing values

# Initialize and train a Random Forest Classifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Make predictions on the test set
y_pred = clf.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print("Accuracy:", accuracy)
print("Classification Report:\n", report)

# Save the trained model to a file
model_path = "cry_detection_model.pkl"
joblib.dump(clf, model_path)

# Fit the model with feature names
clf.feature_names = feature_vector 


Accuracy: 1.0
Classification Report:
               precision    recall  f1-score   support

         0.0       1.00      1.00      1.00        23
         1.0       1.00      1.00      1.00        13

    accuracy                           1.00        36
   macro avg       1.00      1.00      1.00        36
weighted avg       1.00      1.00      1.00        36



In [97]:
sklearn.metrics.confusion_matrix(y_test, y_pred)

array([[23,  0],
       [ 0, 13]], dtype=int64)

In [98]:
import numpy as np
from scipy.io import wavfile
from scipy.signal import spectrogram
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
import joblib

# Function to extract features from an audio file
def extract_features(audio_path):
    # Sample rate and other parameters (should match the training parameters)
    fs = 1000
    nperseg = 256
    noverlap = 128

    # Read the audio file
    fs, audio_signal = wavfile.read(audio_path)

    # Compute STFT
    f, t, Sxx = spectrogram(audio_signal, fs=fs, nperseg=nperseg, noverlap=noverlap)

    # Calculate the 20 features from STFT
    tf_mean = np.mean(Sxx)
    tf_std = np.std(Sxx)
    
    tma = np.max(Sxx, axis=0)
    tma_max = np.max(tma)
    tma_min = np.min(tma)
    tma_mean = np.mean(tma)
    tma_std = np.std(tma)
    tma_skewness = skew(tma)
    tma_kurt = kurtosis(tma)
        
    fma = np.max(Sxx, axis=1)
    fma_max = np.max(fma)
    fma_min = np.min(fma)
    fma_mean = np.mean(fma)
    fma_std = np.std(fma)
    fma_skewness = skew(fma)
    fma_kurt = kurtosis(fma)
        
    fsda = np.std(Sxx, axis=1)
    fsda_max = np.max(fsda)
    fsda_min = np.min(fsda)
    fsda_mean = np.mean(fsda)
    fsda_std = np.std(fsda)
    fsda_skewness = skew(fsda)
    fsda_kurt = kurtosis(fsda)

    # Determine if it's a cry or non-cry sound based on file name or other criteria
    is_cry = 1 if "cry" in audio_file else 0

    # Combine features into a feature vector
    feature_vector = np.array([ tf_mean, tf_std,
                               tma_max, tma_min, tma_mean, tma_std, tma_skewness, tma_kurt,
                               fma_max, fma_min, fma_mean, fma_std, fma_skewness, fma_kurt, 
                               fsda_max, fsda_min, fsda_mean, fsda_std, fsda_skewness, fsda_kurt])

    # Append feature vector and label to lists
    features.append(feature_vector)
    labels.append(is_cry)
    return feature_vector


# Path to the new audio clip 
new_audio_path = r"D:\infant cry detecting\dev\baby_cry\baby_cry_detection\data\903 - Baby laugh\laugh_1.m4a_9.wav"
#r"D:\infant cry detecting\dev\baby_cry\baby_cry_detection\data\new"
# Extract features from the new audio clip
new_feature_vector = extract_features(new_audio_path)

# Make a prediction using the trained model
prediction = clf.predict([new_feature_vector])
print(prediction)
# Map prediction to class label (you can define a mapping if needed)
#class_label = "Cry Sound" if prediction == 1 else "Non-Cry Sound"

# Print the result
#print(f"The audio clip is classified as: {class_label}")
# Interpret the prediction
if prediction[0] == 1:
    print("The new audio clip contains a cry sound.")
else:
    print("The new audio clip does not contain a cry sound.")

[0.]
The new audio clip does not contain a cry sound.


C:\Users\Admin\anaconda3\envs\cry_det\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [99]:
# Folder containing the audio clips you want to classify
folder_path = r"D:\infant cry detecting\dev\baby_cry\baby_cry_detection\data\New folder"

# List all audio files in the folder
audio_files = [f for f in os.listdir(folder_path) if f.endswith(".wav")]

# Classify each audio clip in the folder
for audio_file in audio_files:
    # Get the full path to the audio file
    audio_path = os.path.join(folder_path, audio_file)

    # Extract features from the audio clip
    new_feature_vector = extract_features(audio_path)

    # Make a prediction using the trained model
    prediction = clf.predict([new_feature_vector])

    # Interpret the prediction
    if prediction[0] == 1:
        print(f" '{audio_file}' - a cry sound.")
    else:
        print(f"'{audio_file}' - not a cry sound.")


C:\Users\Admin\anaconda3\envs\cry_det\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\Admin\anaconda3\envs\cry_det\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\Admin\anaconda3\envs\cry_det\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\Admin\anaconda3\envs\cry_det\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\Admin\anaconda3\envs\cry_det\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.w

 'Louise_01.m4a_0.wav' - a cry sound.
 'Louise_01.m4a_1.wav' - a cry sound.
 'Louise_01.m4a_10.wav' - a cry sound.
 'Louise_01.m4a_11.wav' - a cry sound.
 'Louise_01.m4a_12.wav' - a cry sound.
 'Louise_01.m4a_13.wav' - a cry sound.
 'Louise_01.m4a_14.wav' - a cry sound.
 'Louise_01.m4a_2.wav' - a cry sound.
 'Louise_01.m4a_3.wav' - a cry sound.
 'Louise_01.m4a_4.wav' - a cry sound.
 'Louise_01.m4a_5.wav' - a cry sound.
 'Louise_01.m4a_6.wav' - a cry sound.
 'Louise_01.m4a_7.wav' - a cry sound.
 'Louise_01.m4a_8.wav' - a cry sound.
 'Louise_01.m4a_9.wav' - a cry sound.


C:\Users\Admin\anaconda3\envs\cry_det\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\Admin\anaconda3\envs\cry_det\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\Admin\anaconda3\envs\cry_det\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\Admin\anaconda3\envs\cry_det\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\Admin\anaconda3\envs\cry_det\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.w

 'margot.m4a_0.wav' - a cry sound.
 'margot.m4a_1.wav' - a cry sound.
 'margot.m4a_10.wav' - a cry sound.
 'margot.m4a_11.wav' - a cry sound.
 'margot.m4a_12.wav' - a cry sound.
 'margot.m4a_13.wav' - a cry sound.
 'margot.m4a_14.wav' - a cry sound.
 'margot.m4a_15.wav' - a cry sound.
 'margot.m4a_16.wav' - a cry sound.
 'margot.m4a_17.wav' - a cry sound.
 'margot.m4a_18.wav' - a cry sound.
 'margot.m4a_19.wav' - a cry sound.
 'margot.m4a_2.wav' - a cry sound.
 'margot.m4a_20.wav' - a cry sound.


C:\Users\Admin\anaconda3\envs\cry_det\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\Admin\anaconda3\envs\cry_det\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\Admin\anaconda3\envs\cry_det\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\Admin\anaconda3\envs\cry_det\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\Admin\anaconda3\envs\cry_det\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.w

...

C:\Users\Admin\anaconda3\envs\cry_det\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\Admin\anaconda3\envs\cry_det\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\Admin\anaconda3\envs\cry_det\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\Admin\anaconda3\envs\cry_det\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\Admin\anaconda3\envs\cry_det\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.w

'silence.wav_92.wav' - not a cry sound.
'silence.wav_93.wav' - not a cry sound.
'silence.wav_94.wav' - not a cry sound.
'silence.wav_95.wav' - not a cry sound.
'silence.wav_96.wav' - not a cry sound.
'silence.wav_97.wav' - not a cry sound.
'silence.wav_98.wav' - not a cry sound.
'silence.wav_99.wav' - not a cry sound.
 'V_2017-04-01+08_06_22=0_30.mp3_0.wav' - a cry sound.
 'V_2017-04-01+08_06_22=0_30.mp3_1.wav' - a cry sound.
 'V_2017-04-01+08_06_22=0_30.mp3_10.wav' - a cry sound.
 'V_2017-04-01+08_06_22=0_30.mp3_11.wav' - a cry sound.
 'V_2017-04-01+08_06_22=0_30.mp3_12.wav' - a cry sound.
 'V_2017-04-01+08_06_22=0_30.mp3_13.wav' - a cry sound.


C:\Users\Admin\anaconda3\envs\cry_det\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\Admin\anaconda3\envs\cry_det\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\Admin\anaconda3\envs\cry_det\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\Admin\anaconda3\envs\cry_det\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\Admin\anaconda3\envs\cry_det\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.w

 'V_2017-04-01+08_06_22=0_30.mp3_14.wav' - a cry sound.
 'V_2017-04-01+08_06_22=0_30.mp3_15.wav' - a cry sound.
 'V_2017-04-01+08_06_22=0_30.mp3_16.wav' - a cry sound.
 'V_2017-04-01+08_06_22=0_30.mp3_17.wav' - a cry sound.
 'V_2017-04-01+08_06_22=0_30.mp3_18.wav' - a cry sound.
 'V_2017-04-01+08_06_22=0_30.mp3_19.wav' - a cry sound.
 'V_2017-04-01+08_06_22=0_30.mp3_2.wav' - a cry sound.
 'V_2017-04-01+08_06_22=0_30.mp3_20.wav' - a cry sound.
 'V_2017-04-01+08_06_22=0_30.mp3_21.wav' - a cry sound.
 'V_2017-04-01+08_06_22=0_30.mp3_22.wav' - a cry sound.
 'V_2017-04-01+08_06_22=0_30.mp3_23.wav' - a cry sound.
 'V_2017-04-01+08_06_22=0_30.mp3_24.wav' - a cry sound.
 'V_2017-04-01+08_06_22=0_30.mp3_25.wav' - a cry sound.
 'V_2017-04-01+08_06_22=0_30.mp3_3.wav' - a cry sound.
 'V_2017-04-01+08_06_22=0_30.mp3_4.wav' - a cry sound.
 'V_2017-04-01+08_06_22=0_30.mp3_5.wav' - a cry sound.
 'V_2017-04-01+08_06_22=0_30.mp3_6.wav' - a cry sound.
 'V_2017-04-01+08_06_22=0_30.mp3_7.wav' - a cry sound

C:\Users\Admin\anaconda3\envs\cry_det\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\Admin\anaconda3\envs\cry_det\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\Admin\anaconda3\envs\cry_det\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\Admin\anaconda3\envs\cry_det\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\Admin\anaconda3\envs\cry_det\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.w

In [100]:
from sklearn.impute import SimpleImputer  # Import SimpleImputer for handling missing values
from sklearn.svm import SVC


In [101]:
import os
import numpy as np
from scipy.io import wavfile
from scipy.signal import spectrogram
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from scipy.stats import skew, kurtosis


In [102]:
# Create a DataFrame to hold the features and labels
column_names = ['IsCry', 'TF_Mean', 'TF_Std', 'TMA_Max', 'TMA_Min', 'TMA_Mean', 'TMA_Std', 'TMA_Skewness', 'TMA_Kurtosis',
                'FMA_Max', 'FMA_Min', 'FMA_Mean', 'FMA_Std', 'FMA_Skewness', 'FMA_Kurtosis',
                'FSDA_Max', 'FSDA_Min', 'FSDA_Mean', 'FSDA_Std', 'FSDA_Skewness', 'FSDA_Kurtosis']

df = pd.DataFrame(features, columns=column_names)

# Shuffle the DataFrame
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

# Split the data into training and testing sets (80% train, 20% test)
X = df.drop('IsCry', axis=1)
y = df['IsCry']

# Handle missing values in X using SimpleImputer (replace NaNs with the mean of each feature)
imputer = SimpleImputer(strategy='mean')
X = imputer.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train a Support Vector Machine (SVM) Classifier
clf = SVC(kernel='linear', C=1.0, random_state=42)

# Convert the target variable to integers (0 for non-cry, 1 for cry)
y = y.astype(int)

clf.fit(X_train, y_train)

# Make predictions on the test set
y_pred = clf.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print("Accuracy:", accuracy)
print("Classification Report:\n", report)

# Now, for testing a new audio clip:
new_audio_path = r"D:\infant cry detecting\dev\baby_cry\baby_cry_detection\data\903 - Baby laugh\laugh_1.m4a_9.wav"

# Read the new audio file
fs, new_audio_signal = wavfile.read(new_audio_path)

# Compute STFT for the new audio clip (similar to the training data)
f, t, Sxx = spectrogram(new_audio_signal, fs=fs, nperseg=nperseg, noverlap=noverlap)

# Calculate the 20 features from STFT
feature_vector = extract_features(new_audio_path)


# Here, we assume you have a function extract_features(Sxx) that computes the features
# feature_vector = extract_features(Sxx)

# Handle missing values in the feature vector using SimpleImputer
feature_vector = imputer.transform([feature_vector])

# Use the trained SVM classifier to predict the label for the new audio clip
predicted_label = clf.predict(feature_vector)

# Display the predicted label (1 for cry, 0 for non-cry)
print("Predicted Label for the New Audio Clip:", predicted_label[0])


ValueError: Unknown label type: continuous. Maybe you are trying to fit a classifier, which expects discrete classes on a regression target with continuous values.